# Layer: Gold (Business)
**Project:** Lean Logistics Data Pipeline  
**Business Domain:** E-commerce (Olist Dataset)\
**Table Name:** `dm_customers`

---
## 📑 Notebook Information
| Version | Date | Author | Summary of Changes |
| :--- | :--- | :--- | :--- |
| v1.0 | 2026-02-20 | Tássia Marchito | Consolidated script: Customer and Geolocation join, Business comments, and Tags. |

---
## 🎯 Objectives
This notebook creates the Customer Dimension by enriching customer data with geographic coordinates.
* **Data Enrichment:** Joining `tb_customers` with aggregated `tb_geolocation` from Silver.
* **Data Quality:** Ensuring one unique record per `cd_customer_id`.
* **Governance:** Applying standardized prefixes, column comments, and discovery tags.

In [0]:
from pyspark.sql.functions import col, current_timestamp, avg

In [0]:
# 1. Configurações
source_customers = "cat_tm_services_silver.db_logistics.tb_customers"
source_geo = "cat_tm_services_silver.db_logistics.tb_geolocation"
target_table = "cat_tm_services_gold.db_logistics.dm_customers"

print(f"🚀 Building {target_table}...")

# 2. Preparação da Geolocalização (Agregando para evitar duplicidade de CEP)
df_geo_agg = spark.read.table(source_geo) \
    .groupBy("cd_geolocation_zip_code_prefix") \
    .agg(
        avg("cd_geolocation_lat").alias("vl_latitude"),
        avg("cd_geolocation_lng").alias("vl_longitude")
    )

# 3. Join com Clientes
df_customers = spark.read.table(source_customers)

dm_customers = df_customers.join(
    df_geo_agg, 
    df_customers.cd_customer_zip_code_prefix == df_geo_agg.cd_geolocation_zip_code_prefix, 
    "left"
).select(
    col("cd_customer_id"),
    col("cd_customer_unique_id"),
    col("cd_customer_zip_code_prefix").alias("cd_zip_code"),
    col("nm_customer_city").alias("nm_city"),
    col("nm_customer_state").alias("nm_state"),
    col("vl_latitude"),
    col("vl_longitude")
).withColumn("ts_gold_at", current_timestamp())

# 4. Escrita da Tabela
dm_customers.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(target_table)

# 5. Aplicação de Governança (Tags e Comentários)
print(f"📝 Applying metadata to {target_table}...")

# Tags
spark.sql(f"ALTER TABLE {target_table} SET TAGS ('quality' = 'gold', 'domain' = 'logistics', 'type' = 'dimension')")

# Comentários
spark.sql(f"ALTER TABLE {target_table} ALTER COLUMN cd_customer_id COMMENT 'Unique identifier for the customer order point'")
spark.sql(f"ALTER TABLE {target_table} ALTER COLUMN cd_customer_unique_id COMMENT 'Global unique identifier for the customer'")
spark.sql(f"ALTER TABLE {target_table} ALTER COLUMN cd_zip_code COMMENT 'Customer zip code prefix'")
spark.sql(f"ALTER TABLE {target_table} ALTER COLUMN nm_city COMMENT 'City name'")
spark.sql(f"ALTER TABLE {target_table} ALTER COLUMN nm_state COMMENT 'State abbreviation'")
spark.sql(f"ALTER TABLE {target_table} ALTER COLUMN vl_latitude COMMENT 'Average latitude for the zip code'")
spark.sql(f"ALTER TABLE {target_table} ALTER COLUMN vl_longitude COMMENT 'Average longitude for the zip code'")
spark.sql(f"ALTER TABLE {target_table} ALTER COLUMN ts_gold_at COMMENT 'Timestamp of Gold layer processing'")

# Constraints
spark.sql(f"ALTER TABLE {target_table} ALTER COLUMN cd_customer_id SET NOT NULL")
try:
    spark.sql(f"ALTER TABLE {target_table} ADD CONSTRAINT pk_dm_customers PRIMARY KEY(cd_customer_id) RELY")
except:
    pass

print(f"✅ Table {target_table} is complete!")